### Import required libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')


********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [7]:
features_list =['Num_Enq_non_BANK_last_12M',
'Num_Enq_all_loans_last_9M_3M',
'No of enq in past 90days',
'num_PERSONAL LOAN_enq_last_36M',
'Num_of_PERSONAL LOAN_opened_last9M',
'Num_of_PERSONAL LOAN_opened_last12M',
'highest_sanction_amount_credit_cards',            
'num_CONSUMER LOAN_enq_last_36M',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_outstanding_Balance_PERSONAL LOAN_opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_12M',
'Utilization_all',
'Age_of_oldest_open_account',
'total_emi',
'percentage_ever_delinquent',
'num_CONSUMER LOAN_enq_last_6M_36M',
                         
'month_year',
'90_in_24mob']


In [8]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3021516/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3021516/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 17), (15604, 17), (23406,), (15604,))

In [11]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 17) (4682, 17) (15604, 17)


In [12]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [13]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 7.5,
    'alpha':5.5,
    'gamma':5.0,
    'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 500,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.57091	eval-auc:0.57176
[10]	train-auc:0.63559	eval-auc:0.63865
[20]	train-auc:0.63446	eval-auc:0.63938
[30]	train-auc:0.63721	eval-auc:0.63865
[40]	train-auc:0.64055	eval-auc:0.64372
[50]	train-auc:0.64138	eval-auc:0.64532
[60]	train-auc:0.64114	eval-auc:0.64391
[70]	train-auc:0.64258	eval-auc:0.64494
[80]	train-auc:0.64365	eval-auc:0.64561
[90]	train-auc:0.64487	eval-auc:0.64701
[100]	train-auc:0.64588	eval-auc:0.64835
[110]	train-auc:0.64657	eval-auc:0.64862
[120]	train-auc:0.64753	eval-auc:0.64909
[130]	train-auc:0.64743	eval-auc:0.64877
[140]	train-auc:0.64817	eval-auc:0.64896
[150]	train-auc:0.64846	eval-auc:0.64911
[160]	train-auc:0.64902	eval-auc:0.64920
[170]	train-auc:0.64928	eval-auc:0.64963
[180]	train-auc:0.65007	eval-auc:0.65037
[190]	train-auc:0.64990	eval-auc:0.64989
[200]	train-auc:0.65007	eval-auc:0.64985
[210]	train-auc:0.65095	eval-auc:0.65079
[220]	train-auc:0.65134	eval-auc:0.65099
[230]	train-auc:0.65172	eval-auc:0.65140
[240]	train-auc:0.65219	eva

In [14]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.66, Gini: 0.32, KS: 0.22
Eval AUC: 0.65, Gini: 0.31, KS: 0.25
Test AUC: 0.65, Gini: 0.30, KS: 0.22
OOT AUC: 0.65, Gini: 0.31, KS: 0.23


In [15]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.356113         0.309659        0.477110        0.256527   1873   
9          0.240919         0.232204        0.256522        0.212704   1872   
8          0.200321         0.198416        0.212681        0.185230   1872   
7          0.184196         0.175013        0.185229        0.165813   1873   
6          0.166132         0.157745        0.165810        0.150190   1872   
5          0.145833         0.143638        0.150187        0.137454   1872   
4          0.133476         0.132204        0.137440        0.127638   1873   
3          0.104167         0.122574        0.127632        0.116795   1872   
2          0.088675         0.113492        0.116789        0.108037   1872   
1          0.062467         0.098347        0.108033        0.081330   1873   

        label_good  label_bad  
Decile                         
10            1206        667  
9             1421        451  
8             1497        375  
7             1528        345  
6             1561        311  
5             1599        273  
4             1623        250  
3             1677        195  
2             1706        166  
1             1756        117

In [16]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.343370         0.309244        0.471745        0.257152   1561   
9          0.253846         0.231759        0.257127        0.211886   1560   
8          0.212179         0.197511        0.211863        0.184589   1560   
7          0.178091         0.174133        0.184578        0.164815   1561   
6          0.160897         0.157075        0.164814        0.149847   1560   
5          0.144231         0.143758        0.149844        0.137816   1560   
4          0.135985         0.132630        0.137808        0.127849   1559   
3          0.119718         0.122979        0.127843        0.117300   1562   
2          0.082692         0.113500        0.117298        0.107714   1560   
1          0.078155         0.097982        0.107706        0.081330   1561   

        label_good  label_bad  
Decile                         
10            1025        536  
9             1164        396  
8             1229        331  
7             1283        278  
6             1309        251  
5             1335        225  
4             1347        212  
3             1375        187  
2             1431        129  
1             1439        122

In [17]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.350371         0.306039        0.462774        0.255247   1213   
9          0.231657         0.230012        0.255149        0.210173   1213   
8          0.225062         0.195995        0.210163        0.183968   1213   
7          0.187294         0.173579        0.183901        0.164405   1212   
6          0.169002         0.156704        0.164390        0.149554   1213   
5          0.143446         0.143128        0.149516        0.137342   1213   
4          0.137469         0.132452        0.137338        0.128064   1193   
3          0.119318         0.123368        0.128064        0.117688   1232   
2          0.089035         0.114400        0.117680        0.109663   1213   
1          0.070074         0.099698        0.109649        0.081330   1213   

        label_good  label_bad  
Decile                         
10             788        425  
9              932        281  
8              940        273  
7              985        227  
6             1008        205  
5             1039        174  
4             1029        164  
3             1085        147  
2             1105        108  
1             1128         85

In [18]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
0                           Num_Enq_non_BANK_last_12M       120.0
13                         Age_of_oldest_open_account       100.0
6                highest_sanction_amount_credit_cards        99.0
1                        Num_Enq_all_loans_last_9M_3M        97.0
2                            No of enq in past 90days        75.0
12                                    Utilization_all        71.0
10          Utilization_PERSONAL LOAN_opened_last_12M        58.0
15                         percentage_ever_delinquent        57.0
16                  num_CONSUMER LOAN_enq_last_6M_36M        56.0
4                  Num_of_PERSONAL LOAN_opened_last9M        37.0
8                  num_PL_LE_1_Lac_opened_in_last_12M        35.0
7                      num_CONSUMER LOAN_enq_last_36M        34.0
5                 Num_of_PERSONAL LOAN_opened_last12M        31.0
9   Total_outstanding_Balance_PERSONAL LOAN_opened...        28.0
14        

In [19]:
feature_importance_df

Feature  Importance
0                           Num_Enq_non_BANK_last_12M       120.0
13                         Age_of_oldest_open_account       100.0
6                highest_sanction_amount_credit_cards        99.0
1                        Num_Enq_all_loans_last_9M_3M        97.0
2                            No of enq in past 90days        75.0
12                                    Utilization_all        71.0
10          Utilization_PERSONAL LOAN_opened_last_12M        58.0
15                         percentage_ever_delinquent        57.0
16                  num_CONSUMER LOAN_enq_last_6M_36M        56.0
4                  Num_of_PERSONAL LOAN_opened_last9M        37.0
8                  num_PL_LE_1_Lac_opened_in_last_12M        35.0
7                      num_CONSUMER LOAN_enq_last_36M        34.0
5                 Num_of_PERSONAL LOAN_opened_last12M        31.0
9   Total_outstanding_Balance_PERSONAL LOAN_opened...        28.0
14                                          total_emi        19.0
3                      num_PERSONAL LOAN_enq_last_36M        13.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_12M         9.0

In [20]:
#Save the pred 
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_Bureau_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_Bureau_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_Bureau_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_Bureau_Oot_Pred_9024.csv',index=False)


In [21]:
# Save the model
xgb_model.save_model('LeoPayu_Bureau_90_in_24mob.json')
print('Model Saved')

Model Saved
